Redes Neuronales de Grafos (GNN): para predecir genes relacionados con cáncer.

## Flujo de trabajo y Librerías a usar:

Esto lo redactaré una vez finalice.

In [17]:
import requests
import numpy as np
import networkx as nx
import plotly.graph_objects as go
from sklearn.neighbors import NearestNeighbors

## Carga de genes iniciales:


In [18]:
# Genes relacionados con el cáncer de mama
genes_tnbc = [
    "TP53", "PIK3CA", "RB1", "BRCA1", "PTEN",
    "ATM", "EGFR", "BRAF", "BRCA2", "AKT1",
    "PIK3R1", "KDR", "NF1", "ERBB4", "JAK2",
    "NOTCH1", "TRRAP", "MET", "ALK", "CDKN2A"
]


Expansión en proteinas APLICANDO STRING A CADA GEN

In [15]:


# Función para obtener interacciones de proteínas de la API de STRING
def get_string_interactions(genes, species=9606):
    string_api_url = "https://string-db.org/api/json/network"
    params = {
        "identifiers": "%0d".join(genes),  # Genes separados por salto de línea
        "species": species,
        "required_score": 400,
        "limit": 1000
    }

    response = requests.get(string_api_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error en la solicitud: {response.status_code}")
        return None

# Obtener interacciones de STRING para los genes de interés
interactions = get_string_interactions(genes_tnbc)

# Filtrar las interacciones y evitar impresiones innecesarias
def filter_interactions(interactions):
    if interactions:
        filtered_interactions = []
        for interaction in interactions:
            protein_a = interaction['preferredName_A']
            protein_b = interaction['preferredName_B']
            score = interaction['score']
            if score >= 0.8:
                filtered_interactions.append((protein_a, protein_b, score))
        return filtered_interactions
    return []

# Crear una matriz de interacciones basada en las interacciones filtradas
def create_interaction_matrix(interactions):
    proteins = list(set([i[0] for i in interactions] + [i[1] for i in interactions]))
    n = len(proteins)
    matrix = np.zeros((n, n))

    for interaction in interactions:
        i = proteins.index(interaction[0])
        j = proteins.index(interaction[1])
        matrix[i][j] = interaction[2]

    return matrix, proteins

# Aplicar K-Neighbors para expandir la red, limitando a 1 o 2 genes expandidos
def apply_kneighbors(matrix, proteins, k=2):
    nbrs = NearestNeighbors(n_neighbors=min(k+1, len(proteins)), algorithm='auto').fit(matrix)
    distances, indices = nbrs.kneighbors(matrix)

    expanded_genes = {}
    for i, protein in enumerate(proteins):
        expanded_genes[protein] = [proteins[idx] for idx in indices[i][1:k+1]]

    return expanded_genes

# Crear el grafo utilizando NetworkX
def create_gene_network(expanded_network):
    G = nx.Graph()

    for protein, neighbors in expanded_network.items():
        for neighbor in neighbors:
            G.add_edge(protein, neighbor)

    return G

# Crear una visualización interactiva del grafo utilizando Plotly
def draw_interactive_network(G, initial_genes):
    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []

    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_color = []
    node_size = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        if node in initial_genes:
            node_color.append('red')
        else:
            node_color.append('blue')
        node_size.append(10 + 2 * G.degree[node])  # Tamaño basado en grado de conexión

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Grado de conexión',
                xanchor='left',
                titleside='right'
            )
        )
    )

    node_text = [f'{node}: {G.degree[node]} conexiones' for node in G.nodes()]
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<br>Red de genes iniciales y expandidos',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[dict(
                            text="",
                            showarrow=False,
                            xref="paper", yref="paper"
                        )],
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False))
                    )

    fig.show()

# Filtrar interacciones de alta confianza
filtered_interactions = filter_interactions(interactions)

# Crear la matriz de interacción
interaction_matrix, proteins = create_interaction_matrix(filtered_interactions)

# Obtener genes relacionados usando K-Neighbors, limitando a 1 o 2 genes expandidos por cada gen inicial
expanded_network = apply_kneighbors(interaction_matrix, proteins, k=1)

# Crear el grafo
G = create_gene_network(expanded_network)

# Dibujar el grafo interactivo con Plotly
draw_interactive_network(G, genes_tnbc)


In [22]:
# Función para obtener interacciones de proteínas de la API de STRING
def get_string_interactions(genes, species=9606):
    string_api_url = "https://string-db.org/api/json/network"
    params = {
        "identifiers": "%0d".join(genes),  # Genes separados por salto de línea
        "species": species,
        "required_score": 400,  # Umbral mínimo de puntuación
        "limit": 1000  # Número máximo de interacciones
    }

    response = requests.get(string_api_url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error en la solicitud: {response.status_code}")
        return None

# Filtrar las interacciones y devolver solo aquellas con una puntuación alta
def filter_high_score_interactions(interactions, score_threshold=0.8):
    if interactions:
        high_score_interactions = []
        for interaction in interactions:
            protein_a = interaction['preferredName_A']
            protein_b = interaction['preferredName_B']
            score = interaction['score']
            if score >= score_threshold:
                high_score_interactions.append((protein_a, protein_b, score))
        return high_score_interactions
    return []

# Función para expandir la red de genes a partir de genes iniciales
def expand_gene_network(initial_genes, max_expansion=50):
    all_genes = set(initial_genes)  # Usamos un conjunto para evitar duplicados
    all_interactions = []

    # Realizar la expansión de la red
    genes_to_process = initial_genes.copy()  # Comenzamos con los genes iniciales
    while genes_to_process and len(all_genes) < max_expansion:
        # Obtener las interacciones para los genes actuales
        interactions = get_string_interactions(genes_to_process)

        if interactions:
            filtered_interactions = filter_high_score_interactions(interactions)
            all_interactions.extend(filtered_interactions)

            # Añadir nuevos genes a la red
            for _, protein_b, _ in filtered_interactions:
                all_genes.add(protein_b)

            # Actualizar los genes a procesar, solo genes que aún no han sido procesados
            genes_to_process = [gene for gene in all_genes if gene not in genes_to_process]

    return all_genes, all_interactions

# Crear el grafo utilizando NetworkX
def create_gene_network(interactions, all_genes):
    G = nx.Graph()

    # Agregar nodos para todos los genes
    for gene in all_genes:
        G.add_node(gene)

    # Crear las interacciones entre los genes
    for interaction in interactions:
        protein_a, protein_b, score = interaction
        if protein_a in all_genes and protein_b in all_genes:
            G.add_edge(protein_a, protein_b, weight=score)

    return G

# Crear una visualización interactiva del grafo utilizando Plotly
def draw_interactive_network(G):
    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []

    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = []
    node_y = []
    node_color = []
    node_size = []

    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

        # Colorear todos los nodos de la red (puedes agregar una lógica extra si deseas colores diferentes)
        node_color.append('blue')
        node_size.append(10 + 2 * G.degree[node])  # Tamaño basado en grado de conexión

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            color=node_color,
            size=node_size,
            colorbar=dict(
                thickness=15,
                title='Grado de conexión',
                xanchor='left',
                titleside='right'
            )
        )
    )

    node_text = [f'{node}: {G.degree[node]} conexiones' for node in G.nodes()]
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='<br>Red de genes más conectados',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=0, l=0, r=0, t=40),
                        annotations=[dict(
                            text="",
                            showarrow=False,
                            xref="paper", yref="paper"
                        )],
                        xaxis=dict(showgrid=False, zeroline=False),
                        yaxis=dict(showgrid=False, zeroline=False))
                    )

    fig.show()

# Expansión de la red de genes
expanded_genes, expanded_interactions = expand_gene_network(genes_tnbc)

# Crear el grafo con los genes más conectados
G = create_gene_network(expanded_interactions, expanded_genes)

# Dibujar el grafo interactivo
draw_interactive_network(G)
